In [11]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
import os
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd
from sql2gee import SQL2GEE

In [2]:
ee.Initialize()

fusion_table = 'ft:11VR9dNmaDJj33zkB0_wT1KhuN77hbSAO4VYvk6Xq'
fusion_table_SQL2GEE = '"ft:11VR9dNmaDJj33zkB0_wT1KhuN77hbSAO4VYvk6Xq"'
countries_gaul = ee.FeatureCollection(fusion_table)

# AREA OF INTEREST
region_L1 = ee.Geometry.Polygon([[[-30, -40], [65, -30], [65, 40], [-30, 40]]])
region_json= [[-30, -40], [65, -30], [65, 40], [-30, 40]]

In [3]:
sql_0 = 'select COUNT(name) from ' + fusion_table_SQL2GEE 
q_0 = SQL2GEE(sql_0)
q_0.response

87

In [4]:
sql_1 = 'select * from ' + fusion_table_SQL2GEE + ' WHERE name = Syria'
q_1 = SQL2GEE(sql_1)
#q_1.response

# <span style="color:red">INPUTS</span> 

In [53]:
subregions = {
  'na': 'Northern Africa',
  'ma': 'Middle Africa',
  'wa': 'Western Africa', 
  'ea': 'Eastern Africa',
  'sa': 'Southern Africa', 
  'ws': 'Western Asia',
  'ss': 'Southern Asia'
}

#name_asset = 'AET_Annual'
#name_asset = 'AGBP_Annual'
#name_asset = 'T_Annual'
#name_asset = 'WPbm'
name_asset = 'WPnb'
radice_dataset = (name_asset.split("_")[0]) #.lower()
radice_dataset

'WPnb'

In [66]:
# Only change
year = 2016


In [67]:
radice = 'projects/fao-wapor/'
#raster_name = radice + name_asset + "/" + radice_dataset + "-" + str(year)
#raster_name = radice + name_asset + "/" + radice_dataset + "_Annual-" + str(year)
raster_name = radice + name_asset + "/" + radice_dataset + "-" + str(year)
print(raster_name)
raster_stats = ee.Image(raster_name).toFloat()
scale_calc = ee.Image(raster_stats).projection().nominalScale().getInfo()
print('Image scale: ', scale_calc)    
for code,subregion in subregions.iteritems():    
    selected_subregion = subregions[code]    
    filtered = countries_gaul.filter(ee.Filter.eq('subregion', selected_subregion))
    country_number = filtered.size().getInfo()     
    prefix = raster_name.split("/")[2] 
    
    means = raster_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = scale_calc,
    )

    reducers = ee.Reducer.minMax().combine(
        reducer2 = ee.Reducer.sum(),
        sharedInputs = True
    )

    minMaxStds = raster_stats.reduceRegions(
                collection = filtered,
                reducer = reducers,
                scale = scale_calc,
    )
    
    features_m = means.getInfo()['features']
    features_mms = minMaxStds.getInfo()['features']
    df_m = pd.DataFrame(data=features_m[0:], columns = features_m[0])
    df_mms = pd.DataFrame(data=features_mms[0:], columns = features_mms[0])
    serie_m = df_m['properties'].apply(pd.Series)
    serie_mms = df_mms['properties'].apply(pd.Series)
    df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])
    df_mms = pd.DataFrame(serie_mms[['fid', 'min', 'max','sum']])
    df_stats = df_m.set_index('fid').join(df_mms.set_index('fid'), lsuffix='_df_m', rsuffix='_df_mms')
    df_stats.index = df_stats.index.astype(int)
    df_stats['year'] = year
    df_stats['year'] = df_stats['year'].astype(int)
    df_stats['gaul_code'] = df_stats['gaul_code'].astype(int)
    df_stats['gee_asset'] = name_asset
    for col in ['name', 'region', 'subregion', 'gee_asset']:           
        df_stats[col] = df_stats[col].astype('str')
    file_name = name_asset + "_" + str(year) + "_" + code 
    print(file_name)
    df_stats.to_csv('stats/'+ radice_dataset.lower() + "_stats/" + file_name + '.csv')

projects/fao-wapor/WPnb/WPnb-2016
('Image scale: ', 248.48100621908284)
WPnb_2016_ma
WPnb_2016_wa
WPnb_2016_ss
WPnb_2016_na
WPnb_2016_ea
WPnb_2016_ws
WPnb_2016_sa


In [86]:
import glob
import os.path

radice_dataset_l = radice_dataset.lower()
path ='stats/' + radice_dataset_l + '_stats'
print path
fname = 'stats/' + radice_dataset_l + '_annual.csv'

allFiles = glob.glob(path + "/*.csv")
print len(allFiles)
if len(allFiles)==49:
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0)
        list_.append(df)
    frame = pd.concat(list_)
    frame['fid'] = frame['fid'].astype(int)
    frame = frame.set_index('fid')

    try:
        os.remove(fname)
    except OSError:
        pass
    frame.to_csv(fname)
else:
    print "Not completed"

stats/agbp_stats
49
